# day 7

https://adventofcode.com/7/day/7

In [ ]:
import logging
import logging.config
import os

import yaml

In [ ]:
with open('../logging.yaml') as fp:
    logging_config = yaml.load(fp, Loader=yaml.FullLoader)

logging.config.dictConfig(logging_config)

In [ ]:
FNAME = os.path.join('data', 'day07.txt')

LOGGER = logging.getLogger('day07')

## part 1

### problem statement:

#### loading data

In [ ]:
test_data ="""32T3K 765
T55J5 684
KK677 28
KTJJT 220
QQQJA 483"""

In [ ]:
def parse_data(s: str) -> list[list[str | int]]:
    o = []
    for line in s.split('\n'):
        hand_str, bid = line.strip().split(" ")
        bid = int(bid)
        o.append([hand_str, bid])
    return o

In [ ]:
def load_data(fname=FNAME):
    with open(fname) as fp:
        return fp.read().strip()

In [ ]:
parse_data(test_data)

#### function def

In [ ]:
import collections
import enum

class Card(enum.IntEnum):
    X = 0  # stand-in for joker
    _2 = 2
    _3 = 3
    _4 = 4
    _5 = 5
    _6 = 6
    _7 = 7
    _8 = 8
    _9 = 9
    T = 10
    J = 11
    Q = 12
    K = 13
    A = 14

card_lookup = {'A': Card.A,
               '2': Card._2,
               '3': Card._3,
               '4': Card._4,
               '5': Card._5,
               '6': Card._6,
               '7': Card._7,
               '8': Card._8,
               '9': Card._9,
               'T': Card.T,
               'J': Card.J,
               'Q': Card.Q,
               'K': Card.K,
               'X': Card.X, }

class HandType(enum.IntEnum):
    HIGH_CARD = 1
    ONE_PAIR = 2
    TWO_PAIR = 3
    THREE_OF_A_KIND = 4
    FULL_HOUSE = 5
    FOUR_OF_A_KIND = 6
    FIVE_OF_A_KIND = 7

class Hand:
    def __init__(self, hand_str: str):
        self.hand_str = hand_str

    @property
    def cards(self) -> list[Card]:
        return [card_lookup[c] for c in self.hand_str]

    @property
    def hand_type(self) -> HandType:
        hand_str_to_score = self.hand_str.replace('X', '')
        num_jokers = len(self.hand_str) - len(hand_str_to_score)

        c = collections.Counter(hand_str_to_score)
        cts = tuple(sorted(list(c.values())))

        if num_jokers == 0:
            cts_map = {(5,): HandType.FIVE_OF_A_KIND,
                       (1, 4): HandType.FOUR_OF_A_KIND,
                       (2, 3): HandType.FULL_HOUSE,
                       (1, 1, 3): HandType.THREE_OF_A_KIND,
                       (1, 2, 2): HandType.TWO_PAIR,
                       (1, 1, 1, 2): HandType.ONE_PAIR,
                       (1, 1, 1, 1, 1): HandType.HIGH_CARD}
        elif num_jokers == 1:
            cts_map = {(4,): HandType.FIVE_OF_A_KIND,
                       (1, 3): HandType.FOUR_OF_A_KIND,
                       (2, 2): HandType.FULL_HOUSE,
                       (1, 1, 2): HandType.THREE_OF_A_KIND,
                       (1, 1, 1, 1): HandType.ONE_PAIR, }
        elif num_jokers == 2:
            cts_map = {(3,): HandType.FIVE_OF_A_KIND,
                       (1, 2): HandType.FOUR_OF_A_KIND,
                       (1, 1, 1): HandType.THREE_OF_A_KIND, }
        elif num_jokers == 3:
            cts_map = {(2,): HandType.FIVE_OF_A_KIND,
                       (1, 1): HandType.FOUR_OF_A_KIND, }
        elif num_jokers == 4:
            cts_map = {(1,): HandType.FIVE_OF_A_KIND, }
        elif num_jokers == 5:
            cts_map = {tuple(): HandType.FIVE_OF_A_KIND,}
        else:
            raise ValueError("didn't expect to get here")
        return cts_map[cts]

    def __lt__(self, other: 'Hand') -> bool:
        return [self.hand_type, self.cards] < [other.hand_type, other.cards]

In [ ]:
h1 = Hand('32T3K')
h2 = Hand('T55J5')
h3 = Hand('KK677')
h4 = Hand('KTJJT')
h5 = Hand('QQQJA')

assert h1.hand_type is HandType.ONE_PAIR
assert h2.hand_type is HandType.THREE_OF_A_KIND
assert h3.hand_type is HandType.TWO_PAIR
assert h4.hand_type is HandType.TWO_PAIR
assert h5.hand_type is HandType.THREE_OF_A_KIND

assert h1 < h2
assert h1 < h3
assert h1 < h4
assert h1 < h5

assert h4 < h2
assert h4 < h3
assert h4 < h5

assert h3 < h2
assert h3 < h5

assert h2 < h5

In [ ]:
assert Hand('XXXXX').hand_type is HandType.FIVE_OF_A_KIND
assert Hand('QXXQ2').hand_type is HandType.FOUR_OF_A_KIND
assert Hand('XKKK2').hand_type is HandType.FOUR_OF_A_KIND
assert Hand('QQQQ2').hand_type is HandType.FOUR_OF_A_KIND
assert Hand('XKKK2') < Hand('QQQQ2')

In [ ]:
def q_1(data):
    game = parse_data(data)
    game = [[Hand(hand_str), bid] for (hand_str, bid) in game]
    game = sorted(game, key=lambda x: x[0])
    score = sum(rnk * bid for (rnk, (hand, bid)) in enumerate(game, start=1))
    return score

In [ ]:
q_1(test_data)

#### tests

In [ ]:
def test_q_1():
    LOGGER.setLevel(logging.DEBUG)
    assert q_1(test_data) == 6440
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_1()

#### answer

In [ ]:
q_1(load_data())

## part 2

### problem statement:

#### function def

In [ ]:
def q_2(data):
    return q_1(data.replace('J', 'X'))

#### tests

In [ ]:
def test_q_2():
    LOGGER.setLevel(logging.DEBUG)
    assert q_2(test_data) == 5905
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_2()

#### answer

In [ ]:
q_2(load_data())

fin